# Setup

In [ ]:
import getpass
import IPython.display
import json
import os
import requests
import pandas as pd 
import tarfile
from pyngrok import ngrok

from app_gradio import app
from question_answer.answer import Pipeline

In [ ]:
!python3 training/wab.py --fetch

# Serve and Deploy
- Local/Local 
- Cloud Server/Cloud Server

In [ ]:
qa = Pipeline()

In [ ]:
example_img = "question_answer/tests/support/images/img.jpg"
example_question = "question_answer/tests/support/questions/question.txt"

print(qa.predict(example_img, example_question))

In [ ]:
frontend = app.make_frontend(qa.predict, flagging=True)

In [ ]:
frontend.launch(share=True, width="100%")

In [ ]:
%env API_URL={frontend.share_url + "/api"}

In [ ]:
response, = ! \
  (echo -n '{ "data": ["data:image/jpg;base64,'$(base64 -w0 -i question_answer/tests/support/images/img.jpg)'", "data:question/str;str,'$(cat question_answer/tests/support/questions/question.txt)'"] }') \
  | curl -s -X POST "${API_URL}/predict" -H 'Content-Type: application/json' -d @-
  
response

In [ ]:
print(json.loads(response)["data"][0])

In [ ]:
frontend.close()

# Serverless Backend (AWS Lambda)

## Build container image

In [ ]:
os.environ["LAMBDA_NAME"] = "admirer-backend"

In [ ]:
!docker build -t $LAMBDA_NAME . --file api_serverless/Dockerfile #--no-cache

In [ ]:
# export LAMBDA_NAME=admirer-backend
# docker run -p 9000:8080 $LAMBDA_NAME\:latest

In [ ]:
!curl -XPOST \
  "http://localhost:9000/2015-03-31/functions/function/invocations" \
  -d '{"image_url": "question_answer/tests/support/images/img.jpg", "question": "What color is my hair"}'

## Upload to the container registry

In [ ]:
# aws configure

In [ ]:
aws_account_id, = !aws sts get-caller-identity \
  --query "Account"
aws_region, = !aws configure get region 

os.environ["AWS_REGION"] = aws_region
os.environ["AWS_ACCOUNT_ID"] = aws_account_id.strip('"')

!echo $AWS_ACCOUNT_ID
!echo $AWS_REGION

In [ ]:
os.environ["ECR_URI"] = ".".join(
    [os.environ["AWS_ACCOUNT_ID"], "dkr", "ecr", os.environ["AWS_REGION"], "amazonaws.com"])

!echo $ECR_URI

In [ ]:
!aws ecr get-login-password --region $AWS_REGION \
  | docker login --username AWS --password-stdin $ECR_URI

In [ ]:
!aws ecr create-repository \
  --repository-name $LAMBDA_NAME \
  --image-scanning-configuration scanOnPush=true --image-tag-mutability MUTABLE \
  | jq -C

In [ ]:
os.environ["IMAGE_URI"] = "/".join([os.environ["ECR_URI"], os.environ["LAMBDA_NAME"]])

In [ ]:
!docker tag $LAMBDA_NAME\:latest $IMAGE_URI\:latest

In [ ]:
!docker push $IMAGE_URI\:latest

## Create a Lambda function

In [ ]:
os.environ["LAMBDA_ROLE_NAME"] = "lambda-role"

In [ ]:
!aws iam create-role \
  --role-name $LAMBDA_ROLE_NAME \
  --assume-role-policy-document '{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]}' \
  | jq -C

In [ ]:
lambda_role_arn, = !aws iam get-role --role-name $LAMBDA_ROLE_NAME --output json | jq -r '.Role.Arn'
lambda_role_arn = lambda_role_arn.strip('"')

os.environ["LAMBDA_ROLE_ARN"] = lambda_role_arn
!echo $LAMBDA_ROLE_ARN

In [ ]:
# allow this IAM role to execute Lambdas
!aws iam attach-role-policy \
  --role-name $LAMBDA_ROLE_NAME \
  --policy-arn arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole

In [ ]:
# allow this IAM role to write to logs -- required and also important for debugging Lambdas
!aws iam attach-role-policy \
  --role-name $LAMBDA_ROLE_NAME \
  --policy-arn arn:aws:iam::aws:policy/AWSXRayDaemonWriteAccess

In [ ]:
!aws lambda create-function \
  --function-name $LAMBDA_NAME \
  --region $AWS_REGION \
  --package-type Image \
  --code ImageUri=$IMAGE_URI:latest \
  --role $LAMBDA_ROLE_ARN | jq -C

In [ ]:
!aws lambda update-function-configuration \
   --function-name $LAMBDA_NAME \
   --region $AWS_REGION \
   --timeout 60 \
   --memory-size 10240 | jq -C

In [ ]:
!aws lambda invoke \
  --function-name $LAMBDA_NAME \
  --invocation-type RequestResponse \
  --payload '{"image_url": "question_answer/tests/support/images/img.jpg", "question": "What color is my hair"}' \
  --cli-binary-format raw-in-base64-out lambda.out | jq -C

!cat lambda.out

## Add an HTTP endpoint with a URL

In [ ]:
!aws lambda create-function-url-config \
  --function-name $LAMBDA_NAME \
  --auth-type NONE \
  --cors '{"AllowOrigins": ["*"], "AllowCredentials": false}' \
  | jq -C

In [ ]:
# Careful here!!!
# """
!aws lambda add-permission \
 --function-name $LAMBDA_NAME \
 --action lambda:invokeFunctionUrl \
 --statement-id "open-access" \
 --principal "*" \
 --function-url-auth-type NONE | jq -C
# """

In [ ]:
lambda_url, = !aws lambda get-function-url-config --function-name $LAMBDA_NAME | jq .FunctionUrl
lambda_url = lambda_url.strip('"')

lambda_url

In [ ]:
image_url = "question_answer/tests/support/images/img.jpg"
question = "What color is my hair"

headers = {"Content-type": "application/json"}
payload = json.dumps({"image_url": image_url, "question": question})

response = requests.post(
  lambda_url, data=payload, headers=headers)
pred = response.json()["pred"]

print(pred)

## Connect AWS with Gradio

In [ ]:
serverless_backend = app.PredictorBackend(url=lambda_url)

In [ ]:
frontend_serverless_backend = app.make_frontend(serverless_backend.run, flagging=True)
frontend_serverless_backend.launch(share=True)

In [ ]:
frontend_serverless_backend.close()

# Serving through Ngrok

In [ ]:
frontend = frontend #frontend_serverless_backend
frontend.local_url

In [ ]:
!curl -X POST {frontend.local_url}api/predict

In [ ]:
config_file = ngrok.conf.DEFAULT_NGROK_CONFIG_PATH
config_file_exists =  os.path.exists(config_file)
config_file_contents = !cat {config_file}

auth_token_found = config_file_exists \
    and config_file_contents \
    and "authtoken" in config_file_contents[0] \
    and ": exit" not in config_file_contents  # state if interrupted

if not auth_token_found:
    print("Enter your ngrok auth token, which can be copied from https://dashboard.ngrok.com/auth")
    !ngrok authtoken {getpass.getpass()}

In [ ]:
ADMIRER_PORT = frontend.server_port
ADMIRER_PORT

In [ ]:
# https_tunnel = ngrok.connect(ADMIRER_PORT, bind_tls=True)
# print(https_tunnel)

# Load Testing with Locust

## Running the load test

In [ ]:
!locust --locustfile=locust_http_user.py \
  --headless \
  --users=10 \
  --spawn-rate=1 \
  --run-time=2m \
  --host=https://joiajq6syp65ueonto4mswttzu0apfbi.lambda-url.us-west-1.on.aws \
  --html=locust_report.html \
  --csv=locust_report

## Viewing the results

In [ ]:
!ls -lh locust_report*

In [ ]:
IPython.display.HTML("locust_report.html")

## Analyzing load test data programmatically

In [4]:
csv_path = "locust_report_stats_history.csv"
results = pd.read_csv(csv_path)
results["Timestamp"] = pd.to_datetime(results["Timestamp"], unit="s")
results.tail()

,Timestamp,User Count,Type,Name,Requests/s,Failures/s,50%,66%,75%,80%,...,99.9%,99.99%,100%,Total Request Count,Total Failure Count,Total Median Response Time,Total Average Response Time,Total Min Response Time,Total Max Response Time,Total Average Content Size
106,2022-10-20 03:30:54,10,NaN,Aggregated,0.7,0.0,13000.0,13000.0,14000.0,17000.0,...,44000.0,44000.0,44000.0,52,0,13000.0,17902.302880,10327.992856,43529.908691,16.0
107,2022-10-20 03:30:55,10,NaN,Aggregated,0.7,0.0,13000.0,13000.0,14000.0,17000.0,...,44000.0,44000.0,44000.0,52,0,13000.0,17902.302880,10327.992856,43529.908691,16.0
108,2022-10-20 03:30:56,10,NaN,Aggregated,0.7,0.0,13000.0,13000.0,14000.0,17000.0,...,44000.0,44000.0,44000.0,52,0,13000.0,17902.302880,10327.992856,43529.908691,16.0
109,2022-10-20 03:30:57,10,NaN,Aggregated,1.0,0.0,13000.0,13000.0,13000.0,17000.0,...,44000.0,44000.0,44000.0,53,0,13000.0,17774.222529,10327.992856,43529.908691,16.0
110,2022-10-20 03:30:58,10,NaN,Aggregated,0.9,0.0,13000.0,13000.0,13000.0,17000.0,...,44000.0,44000.0,44000.0,54,0,13000.0,17639.795922,10327.992856,43529.908691,16.0


In [5]:
request_columns = ["Total Request Count", "Total Failure Count", "User Count"]
results.plot(x="Timestamp", y=request_columns, subplots=True, sharey=True);

In [6]:
response_columns = ["Total Average Response Time", "Total Max Response Time"]
results.plot(x="Timestamp", y=response_columns);

In [7]:
results.groupby("Total Median Response Time").describe()

User Count                                         \
                                count       mean       std   min   25%   50%   
Total Median Response Time                                                     
0.000000                         41.0   8.658537  2.789353   0.0  10.0  10.0   
13000.000000                     39.0  10.000000  0.000000  10.0  10.0  10.0   
14000.000000                      4.0  10.000000  0.000000  10.0  10.0  10.0   
17000.000000                      3.0  10.000000  0.000000  10.0  10.0  10.0   
39000.000000                      3.0  10.000000  0.000000  10.0  10.0  10.0   
40000.000000                      1.0  10.000000       NaN  10.0  10.0  10.0   
40287.908749                      4.0  10.000000  0.000000  10.0  10.0  10.0   
41000.000000                     16.0  10.000000  0.000000  10.0  10.0  10.0   

                                        Type       ...  \
                             75%   max count mean  ...   
Total Median Response Time                         ...   
0.000000                    10.0  10.0   0.0  NaN  ...   
13000.000000                10.0  10.0   0.0  NaN  ...   
14000.000000                10.0  10.0   0.0  NaN  ...   
17000.000000                10.0  10.0   0.0  NaN  ...   
39000.000000                10.0  10.0   0.0  NaN  ...   
40000.000000                10.0  10.0   0.0  NaN  ...   
40287.908749                10.0  10.0   0.0  NaN  ...   
41000.000000                10.0  10.0   0.0  NaN  ...   

                           Total Max Response Time                \
                                               75%           max   
Total Median Response Time                                         
0.000000                                  0.000000      0.000000   
13000.000000                          43529.908691  43529.908691   
14000.000000                          43529.908691  43529.908691   
17000.000000                          43529.908691  43529.908691   
39000.000000                          43529.908691  43529.908691   
40000.000000                          43529.908691  43529.908691   
40287.908749                          40287.908749  40287.908749   
41000.000000                          43529.908691  43529.908691   

                           Total Average Content Size                         \
                                                count  mean  std   min   25%   
Total Median Response Time                                                     
0.000000                                         41.0   0.0  0.0   0.0   0.0   
13000.000000                                     39.0  16.0  0.0  16.0  16.0   
14000.000000                                      4.0  16.0  0.0  16.0  16.0   
17000.000000                                      3.0  16.0  0.0  16.0  16.0   
39000.000000                                      3.0  16.0  0.0  16.0  16.0   
40000.000000                                      1.0  16.0  NaN  16.0  16.0   
40287.908749                                      4.0  16.0  0.0  16.0  16.0   
41000.000000                                     16.0  16.0  0.0  16.0  16.0   

                                              
                             50%   75%   max  
Total Median Response Time                    
0.000000                     0.0   0.0   0.0  
13000.000000                16.0  16.0  16.0  
14000.000000                16.0  16.0  16.0  
17000.000000                16.0  16.0  16.0  
39000.000000                16.0  16.0  16.0  
40000.000000                16.0  16.0  16.0  
40287.908749                16.0  16.0  16.0  
41000.000000                16.0  16.0  16.0  

[8 rows x 168 columns]